# KDTree sandbox

Using k-d trees to speed up neighbor lookup. The centroids of map polygons go into a [KDTree](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html) which is queried for the $k$ closest polygons to random sampling points, one point at a time.

## Examples from `scikit`

In [6]:
import numpy as np
from sklearn.neighbors import KDTree

In [2]:
rng = np.random.RandomState(0)

X = rng.random_sample((10, 3))  # 10 points in 3 dimensions

In [3]:
X

array([[0.5488135 , 0.71518937, 0.60276338],
       [0.54488318, 0.4236548 , 0.64589411],
       [0.43758721, 0.891773  , 0.96366276],
       [0.38344152, 0.79172504, 0.52889492],
       [0.56804456, 0.92559664, 0.07103606],
       [0.0871293 , 0.0202184 , 0.83261985],
       [0.77815675, 0.87001215, 0.97861834],
       [0.79915856, 0.46147936, 0.78052918],
       [0.11827443, 0.63992102, 0.14335329],
       [0.94466892, 0.52184832, 0.41466194]])

In [7]:
tree = KDTree(X, leaf_size=2)              

In [9]:
tree.data

<MemoryView of 'array' at 0x7ff7b73c6048>

In [16]:
tree.get_arrays()

(array([[0.5488135 , 0.71518937, 0.60276338],
        [0.54488318, 0.4236548 , 0.64589411],
        [0.43758721, 0.891773  , 0.96366276],
        [0.38344152, 0.79172504, 0.52889492],
        [0.56804456, 0.92559664, 0.07103606],
        [0.0871293 , 0.0202184 , 0.83261985],
        [0.77815675, 0.87001215, 0.97861834],
        [0.79915856, 0.46147936, 0.78052918],
        [0.11827443, 0.63992102, 0.14335329],
        [0.94466892, 0.52184832, 0.41466194]]),
 array([8, 3, 0, 4, 9, 5, 1, 7, 6, 2]),
 array([(0, 10, 0, 1.14121853), (0,  5, 0, 0.3923255 ),
        (5, 10, 0, 0.25723308), (0,  2, 1, 0.19277082),
        (2,  5, 1, 0.37598799), (5,  7, 1, 0.09336287),
        (7, 10, 1, 0.14007019)],
       dtype=[('idx_start', '<i8'), ('idx_end', '<i8'), ('is_leaf', '<i8'), ('radius', '<f8')]),
 array([[[0.0871293 , 0.0202184 , 0.07103606],
         [0.11827443, 0.52184832, 0.07103606],
         [0.0871293 , 0.0202184 , 0.64589411],
         [0.11827443, 0.63992102, 0.14335329],
         [0.

### Query for k-nearest neighbors

#### `KDTree.query`

Documentation for [`sklearn.neighbors.KDTree.query()`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html#sklearn.neighbors.KDTree.query).

In [10]:
X[:1]

array([[0.5488135 , 0.71518937, 0.60276338]])

In [11]:
# query takes an array
# retrieve up to 3 nearest neighbors
dist, ind = tree.query(X[:1], k=3)                

In [12]:
# query returns an array of the indices of the closest k neihbors
# to each point (so a 2D array), in this case only one (so it appears nested)
# note that the point itself is returned
print(ind)

[[0 3 1]]


In [13]:
print(dist)

[[0.         0.19662693 0.29473397]]


### Query for neighbors in a certain radius

In [14]:
print(tree.query_radius(X[:1], r=0.3, count_only=True))

[3]


In [15]:
ind = tree.query_radius(X[:1], r=0.3)  
print(ind)  # indices of neighbors within distance 0.3

[array([3, 0, 1])]


### Compute Gaussian density estimate around points

_Might this be used as a crude way to dynamically sample and ensure more uniform coverage of the random sample?_

In [17]:
rng = np.random.RandomState(42)
X = rng.random_sample((100, 3))
tree_gaus = KDTree(X)                
tree_gaus.kernel_density(X[:3], h=0.1, kernel='gaussian')

array([ 97.37160967, 168.6409    , 127.70081271])

## Interplay with `networkx` and `shapely`

[NetworkX](https://networkx.org/) is a package for building and manipulating complex networks in Python.  

[Shapely](https://shapely.readthedocs.io/en/stable/project.html) is a Python package for the manipulation and analysis of geometric objects in the Cartesian plane.  

1. Using the _colliders_ data, a list of objects containing `shapely.geometry.Polygon` objects is populated. Various polygon attributes are exposed, wrapping `shapely.geometry.Polygon` for easier calls. Centroid (for `KDTree`) and coordinates (for `networkx`) properties are most useful.
2. The polygon list is used to create `shapely` objects for the polygons, so that they can be queried for collision with sample points and potential path segments between non-colliding nodes.
3. A `KDTree` is populated with the centroids of the `shapely.geometry.Polygon` objects, in the same order as the poly-containing object list. These indices will be returned by the neighbor query and used for poly lookup.  
4. A random sample of 3-coordinate points is generated and is used to populate a list of points to become graph nodes. The `KDTree` is queried for the $k$ neigbor polygon centroids within a certain _radius_. _Hyperparameter: The choice of radius balances the network graph density (and therefore computational load) and the connectivity of the navigated map. It makes some sense to make the radius the largest polygon dimension (height or width) in the polygon set. TODO: Sketch the rationale._  
5. For each point query, the `KDTree.query` returns the indices of the centroids which matched. Each corresponding polygon is looked up by index and checked for collision. Only non-colliding points are kept.
6. The filtered points are used as graph nodes to populate a new `KDTree` and a `networkx.Graph`.
7. For each node, the closest $k$ are connected if and only if, the `shapely.LineSegment` between them does not cross any of the polygons (no check for the closest here) at a higher lower than the polygon height.